# 第10章 数据聚合与分组运算

## GroupBy机制

In [2]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.582339,1.609926
1,a,two,-0.327789,1.291997
2,b,one,1.083927,-1.300419
3,b,two,-0.204162,-0.255670
4,a,one,-1.263163,-1.058606


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a   -0.724430
b    0.439883
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -0.922751
      two    -0.327789
b     one     1.083927
      two    -0.204162
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-0.922751,-0.327789
b,1.083927,-0.204162


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [11]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [12]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.327789
            2006    1.083927
Ohio        2005   -0.393250
            2006   -1.263163
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.724430,0.614439
b,0.439883,-0.778045


In [14]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.922751  0.275660
     two  -0.327789  1.291997
b    one   1.083927 -1.300419
     two  -0.204162 -0.255670

In [15]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.582339  1.609926
1    a  two -0.327789  1.291997
4    a  one -1.263163 -1.058606
b
  key1 key2     data1     data2
2    b  one  1.083927 -1.300419
3    b  two -0.204162 -0.255670


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.582339  1.609926
4    a  one -1.263163 -1.058606
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.327789  1.291997
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.083927 -1.300419
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.204162 -0.25567


In [18]:
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.083927,-1.300419
3,b,two,-0.204162,-0.255670


In [21]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [22]:
grouped = df.groupby(df.dtypes, axis=1)

In [23]:
grouped

In [25]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.582339  1.609926
1 -0.327789  1.291997
2  1.083927 -1.300419
3 -0.204162 -0.255670
4 -1.263163 -1.058606
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集

In [27]:
df.groupby('key1')['data1']

In [28]:
df.groupby('key1')[['data2']]

In [29]:
df['data1'].groupby(df['key1'])

In [30]:
df[['data2']].groupby(df['key1'])

In [31]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.275660
     two   1.291997
b    one  -1.300419
     two  -0.255670

In [33]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [34]:
s_grouped

In [35]:
s_grouped.mean()

key1  key2
a     one     0.275660
      two     1.291997
b     one    -1.300419
      two    -0.255670
Name: data2, dtype: float64

### 通过字典或Series进行分组

In [36]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [37]:
people

,a,b,c,d,e
Joe,1.205080,1.063688,0.056246,0.937241,-0.106897
Steve,0.178335,-1.001048,-0.479410,-1.028199,2.155178
Wes,0.222836,1.881997,1.495302,2.281552,-0.238783
Jim,-1.942158,-0.801052,0.669375,-1.607967,0.340637
Travis,1.122139,1.662411,0.789315,0.132945,0.550445


In [38]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [39]:
by_column = people.groupby(mapping, axis=1)

In [40]:
by_column.sum()

,blue,red
Joe,0.993487,2.161872
Steve,-1.507610,1.332464
Wes,3.776854,1.866050
Jim,-0.938593,-2.402572
Travis,0.922260,3.334995


In [41]:
map_series = pd.Series(mapping)

In [42]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [43]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,2,3
Jim,2,3
Travis,2,3


### 通过函数进行分组

In [44]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.514241,2.144633,2.220923,1.610826,-0.005042
5,0.178335,-1.001048,-0.479410,-1.028199,2.155178
6,1.122139,1.662411,0.789315,0.132945,0.550445


In [45]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [46]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.222836  1.063688  0.056246  0.937241 -0.238783
  two -1.942158 -0.801052  0.669375 -1.607967  0.340637
5 one  0.178335 -1.001048 -0.479410 -1.028199  2.155178
6 two  1.122139  1.662411  0.789315  0.132945  0.550445

### 根据索引级别分组

In [47]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [48]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)

In [49]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.812841 -1.131861  1.172160  0.056175 -1.096842
1     -1.058748  0.018846 -0.860769  1.287508 -0.328474
2     -1.256895  2.991251 -1.007243 -0.436517  0.426520
3      0.592478  0.156120 -0.059817 -0.312119 -1.174344

In [50]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 数据聚合

In [51]:
df

,key1,key2,data1,data2
0,a,one,-0.582339,1.609926
1,a,two,-0.327789,1.291997
2,b,one,1.083927,-1.300419
3,b,two,-0.204162,-0.255670
4,a,one,-1.263163,-1.058606


In [52]:
grouped = df.groupby('key1')

In [53]:
grouped

In [54]:
grouped['data1'].quantile(0.9)

key1
a   -0.378699
b    0.955118
Name: data1, dtype: float64

In [55]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [56]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.935374,2.668532
b,1.288089,1.044749


In [57]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.724430  0.483605 -1.263163 -0.922751 -0.582339 -0.455064   
b      2.0  0.439883  0.910816 -0.204162  0.117861  0.439883  0.761905   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.327789   3.0  0.614439  1.457593 -1.058606  0.116696  1.291997   
b     1.083927   2.0 -0.778045  0.738749 -1.300419 -1.039232 -0.778045   

                          
           75%       max  
key1                      
a     1.450961  1.609926  
b    -0.516857 -0.255670

### 面向列的多函数应用

In [63]:
tips = pd.read_csv('./examples/tips.csv')

In [66]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [67]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [69]:
grouped = tips.groupby(['day', 'smoker'])

In [71]:
grouped_pct = grouped['tip_pct']

In [72]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [73]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [74]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [77]:
functions = ['count', 'mean', 'max']

In [78]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [79]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [80]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [81]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [82]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [83]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 以“没有行索引”的形式返回聚合数据

In [85]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 apply: 一般性的“拆分-应用-合并”

In [86]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [87]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [88]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [89]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [90]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [91]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [92]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [93]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [94]:
f = lambda x: x.describe()

In [95]:
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

### 禁止分组键

In [96]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

In [97]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

In [98]:
quartiles = pd.cut(frame.data1, 4)

In [99]:
quartiles[:10]

0     (-1.415, 0.112]
1    (-2.948, -1.415]
2     (-1.415, 0.112]
3     (-1.415, 0.112]
4     (-1.415, 0.112]
5     (-1.415, 0.112]
6     (-1.415, 0.112]
7     (-1.415, 0.112]
8      (0.112, 1.639]
9     (-1.415, 0.112]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.948, -1.415] < (-1.415, 0.112] < (0.112, 1.639] < (1.639, 3.166]]

In [100]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [101]:
grouped = frame.data2.groupby(quartiles)

In [102]:
grouped

In [103]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.948, -1.415]",73.0,2.640402,0.080919,-2.115172
"(-1.415, 0.112]",471.0,2.793808,0.026611,-2.537305
"(0.112, 1.639]",410.0,2.619051,0.033617,-2.691419
"(1.639, 3.166]",46.0,3.036008,-0.013293,-1.990814


In [104]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

In [105]:
grouped = frame.data2.groupby(grouping)

In [107]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.640402,0.013241,-2.537305
1,100.0,2.793808,0.136315,-2.252934
2,100.0,2.544648,0.107063,-2.229245
3,100.0,2.680537,0.018007,-2.398462
4,100.0,2.004357,-0.082675,-1.976570
5,100.0,2.517947,0.098604,-2.524480
6,100.0,1.956575,-0.050391,-2.691419
7,100.0,2.619051,-0.003401,-2.246820
8,100.0,1.817754,0.020340,-2.226341


### 示例：用特定于分组的值填充缺失值

In [108]:
s = pd.Series(np.random.randn(8))

In [109]:
s

0   -0.283738
1   -0.907597
2   -0.562517
3   -0.280406
4    0.891964
5   -0.783021
6   -0.840616
7    0.385929
dtype: float64

In [110]:
s[::2]

0   -0.283738
2   -0.562517
4    0.891964
6   -0.840616
dtype: float64

In [111]:
s[::2] = np.nan

In [112]:
s

0         NaN
1   -0.907597
2         NaN
3   -0.280406
4         NaN
5   -0.783021
6         NaN
7    0.385929
dtype: float64

In [113]:
s.fillna(s.mean())

0   -0.396274
1   -0.907597
2   -0.396274
3   -0.280406
4   -0.396274
5   -0.783021
6   -0.396274
7    0.385929
dtype: float64

In [114]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [115]:
group_key = ['East'] * 4 + ['West'] * 4

In [116]:
data = pd.Series(np.random.randn(8), index=states)

In [117]:
data

Ohio         -0.669577
New York     -0.054480
Vermont      -0.152850
Florida       0.763476
Oregon       -0.632380
Nevada        0.233967
California   -0.499729
Idaho         0.428568
dtype: float64

In [118]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [119]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [120]:
data

Ohio         -0.669577
New York     -0.054480
Vermont            NaN
Florida       0.763476
Oregon       -0.632380
Nevada             NaN
California   -0.499729
Idaho              NaN
dtype: float64

In [121]:
data.groupby(group_key).mean()

East    0.013140
West   -0.566054
dtype: float64

In [122]:
fill_mean = lambda g: g.fillna(g.mean())

In [124]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.669577
New York     -0.054480
Vermont       0.013140
Florida       0.763476
Oregon       -0.632380
Nevada       -0.566054
California   -0.499729
Idaho        -0.566054
dtype: float64

In [125]:
fill_values = {'East': 0.5, 'West': -1}

In [126]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [127]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.669577
New York     -0.054480
Vermont       0.500000
Florida       0.763476
Oregon       -0.632380
Nevada       -1.000000
California   -0.499729
Idaho        -1.000000
dtype: float64

### 示例：随机采用和排列

In [128]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']

In [129]:
card_val = (list(range(1, 11)) + [10] * 3) * 4

In [130]:
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

In [131]:
cards = []

In [132]:
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

In [133]:
deck = pd.Series(card_val, index=cards)

In [134]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [135]:
def draw(deck, n=5):
    return deck.sample(n)

In [136]:
draw(deck)

AC     1
QD    10
5H     5
4C     4
4D     4
dtype: int64

In [137]:
get_suit = lambda card: card[-1] # last letter is suit

In [138]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C     2
   5C     5
D  JD    10
   4D     4
H  JH    10
   3H     3
S  5S     5
   AS     1
dtype: int64

In [139]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

2C     2
9C     9
2D     2
5D     5
8H     8
AH     1
3S     3
QS    10
dtype: int64

### 示例：分组加权平均数和相关系数

In [141]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

In [142]:
df

,category,data,weights
0,a,0.281137,0.572291
1,a,1.339562,0.654907
2,a,-0.826976,0.671953
3,a,1.267604,0.598007
4,b,-0.491352,0.029314
5,b,-1.557156,0.481158
6,b,0.059378,0.966832
7,b,-2.432762,0.711434


In [143]:
grouped = df.groupby('category')

In [144]:
grouped

In [145]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [146]:
grouped.apply(get_wavg)

category
a    0.496776
b   -1.113419
dtype: float64

In [147]:
close_px = pd.read_csv('.\examples\stock_px_2.csv', parse_dates=True,
                      index_col=0)

In [148]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [149]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [150]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [152]:
rets = close_px.pct_change().dropna()

In [153]:
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
2003-01-10,0.002725,0.001824,-0.013927,0.000000
2003-01-13,-0.005435,0.008648,-0.004134,-0.001412
2003-01-14,-0.002732,0.010379,0.008993,0.005830
2003-01-15,-0.010959,-0.012506,-0.013713,-0.014426
2003-01-16,0.012465,-0.016282,0.004519,-0.003942


In [154]:
get_year = lambda x: x.year

In [ ]:
by_year 